<a href="https://colab.research.google.com/github/andy-j-block/lymphoma_classifier/blob/master/get_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>